EXPLICACION UNO: Se definen librerías de pandas y numpy, así como de sklearn y mlxtend.
Abrimos la base de datos y armamos una vista corta para asegurarnos de que la información llegó completa.
Verificamos el tamaño, dimensiones, y revisamos el muestreo de la información con la función .shape y .columns e imprimimos las primeras 5 filas de la base de datos.

In [104]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

df=pd.read_csv("VinoTinto.csv")
print(df.shape)
print(df.columns)
print(df.head(5))

(1599, 12)
Index(['acidezFija', 'acidezVolatil', 'acidoCitrico', 'azucarResidual',
       'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'densidad',
       'pH', 'sulfatos', 'alcohol', 'calidad'],
      dtype='object')
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17

EXPLICACION DOS: Aquí separamos el dataset en dos subconjuntos sin superposición, uno para aprender (train) y otro para comprobar(test); se dividen en 80/20,con la funcion de .sample y se agrega la fraccion 0.8.
Imprimimos en consola las dimensiones de train y test, asegurandonos que con la suma de ambas sean el total de la base de datos.

In [105]:
train=df.sample(frac=0.8)
test=df.drop(train.index)
print("Train:",train.shape)
print("Test:",test.shape)
sum=train.shape[0]+test.shape[0]
print("Suma train+test:",sum)
print(train.head(15))

Train: (1279, 12)
Test: (320, 12)
Suma train+test: 1599
      acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
186          7.4           0.39          0.48            2.00     0.082   
1289         7.0           0.60          0.30            4.50     0.068   
1471         6.7           0.70          0.08            3.75     0.067   
205         12.8           0.30          0.74            2.60     0.095   
639          8.9           0.29          0.35            1.90     0.067   
1154         6.6           0.58          0.00            2.20     0.100   
179          8.8           0.61          0.14            2.40     0.067   
1224        12.6           0.39          0.49            2.50     0.080   
116          8.3           0.54          0.28            1.90     0.077   
1406         8.2           0.24          0.34            5.10     0.062   
549          9.0           0.53          0.49            1.90     0.171   
572         10.2           0.24          0.4

EXPLICACION TRES: Dropeamos el dato de calidad, pues será el dato a predecir, y creamos un modelo lineal.
Configuramos la selección hacia adelante,con la función de sfs que forward=True empieza vacío y va agregando variables una a una; k_features=(2,8) buscará el mejor subconjunto de entre 2 y 8 variables.
R2 nos mide la calidad de cada conjunto.
cv=10 usa validación cruzada, repartiendo los datos en 10 partes, entrena 9 y valida en 1, repite y promedia.


In [106]:
Xtrain = train.drop('calidad', axis=1)
ytrain = train['calidad']

linearR = LinearRegression()

sfs_forward = sfs(linearR,k_features=(2, 8),forward=True,floating=False,scoring='r2',cv=10,n_jobs=-1)

sfs_forward = sfs_forward.fit(Xtrain, ytrain)
feat_forward = list(sfs_forward.k_feature_names_)
print("Características seleccionadas (Forward):", feat_forward)

Características seleccionadas (Forward): ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


EXPLICACION CUATRO: Definimos en XtrainForward el feat_forward que solo toma las columnas que el algoritmo Forwrad seleccionó como relevantes.
Con linearR.fit ajustamos de nuevo los coeficientes de beta, pero ahora usando solo las variables forward.
En ypredForward el modelo genera las predicciones de calidad sobre el conjunto de prueba. Como las variables de entrada ahora son distintas, las predicciones pueden diferir respecto al modelo original con todas las variables.
Y por último calculamos el R2 con las predicciones nuevas, esto nos ayuda a medir que tan bien es el subconjunto eelgido por Forward, explicando la variabilidad de la calidad en datos no vistos.
Comparamos el r2Forward contra el R2 del modelo base.

In [107]:
XtrainForward = Xtrain[feat_forward]
XtestForward  = Xtest[feat_forward]

linearR.fit(XtrainForward, ytrain)
ypredForward = linearR.predict(XtestForward)

r2Forward = r2_score(ytest, ypredForward)
print("R² del modelo Forward:", r2Forward)

R² del modelo Forward: 0.466134238113042


EXPLICACION CINCO: Creamos otro sfs pero ahora con forward=False, es decir, backward, este empieza con todas las variables candidatas y va eliminando una a una las que menos aportan.
k_features=(2,5), dice que busque el mejor subconjunto entre 2 y 5 variables.
scoring=r2 revisa la calidad de cada subconjunto y se mide con el coeficiente de determinación.
Luego en el segundo sfs_backward, no empezamos desde las columnas orignales, sino de las que ya se seleccionaron por Forward, a partir de este conjunto el Backward prueba quitando variables de una en una, siempre evaluando el R2 en validación cruzada, hasta llegar al subconjunto mas compacto.
Al final k_feature_names_ nos regresa el nombre exacto de las variables que sobrevivieron al proceso de eliminación.

In [108]:
sfs_backward = sfs(estimator=linearR,k_features=(2, 5),forward=False,floating=False,scoring='r2',cv=10,n_jobs=-1)

sfs_backward = sfs_backward.fit(Xtrain[feat_forward], ytrain)

feat_backward = list(sfs_backward.k_feature_names_)
print("Características seleccionadas (Backward desde Forward):", feat_backward)

Características seleccionadas (Backward desde Forward): ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


EXPLICACION SEIS: Filtramos Xtrain y Xtest para quedarnos solo con las columnas que pasaron al proceso de Backward, esto reduce la dimensión del problema al mejor conjunto.
Reestimamos los coeficientes con el subespacio final. fit() vuelve a resolver mínimos cuadrados, pero ahora en el espacio de características reducido.
Predecimos en test con el modelo compacto, usando el mismo orden y las mismas columnas que en el entrenamiento.
Por último medimos la generalización.

In [109]:
XtrainBackward = Xtrain[feat_backward]
XtestBackward  = Xtest[feat_backward]

linearR.fit(XtrainBackward, ytrain)
ypredBackward = linearR.predict(XtestBackward)

r2Backward = r2_score(ytest, ypredBackward)
print("R² del modelo Backward:", r2Backward)

R² del modelo Backward: 0.45304512202438574
